In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [4]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [5]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [5]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [6]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [8]:
#Standardize the features
X = pipeline.fit_transform(X)

In [9]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [10]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [11]:
#Initialize ANN
classifier = Sequential()

In [12]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [13]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [14]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [15]:
#Let us take a look at our network
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [16]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 3s - loss: 0.6205 - accuracy: 0.6782 - val_loss: 0.4912 - val_accuracy: 0.7950 - 3s/epoch - 11ms/step
Epoch 2/200
225/225 - 1s - loss: 0.4929 - accuracy: 0.7944 - val_loss: 0.4494 - val_accuracy: 0.7950 - 622ms/epoch - 3ms/step
Epoch 3/200
225/225 - 1s - loss: 0.4703 - accuracy: 0.7967 - val_loss: 0.4297 - val_accuracy: 0.8062 - 628ms/epoch - 3ms/step
Epoch 4/200
225/225 - 1s - loss: 0.4577 - accuracy: 0.8032 - val_loss: 0.4183 - val_accuracy: 0.8150 - 619ms/epoch - 3ms/step
Epoch 5/200
225/225 - 1s - loss: 0.4488 - accuracy: 0.8071 - val_loss: 0.4108 - val_accuracy: 0.8225 - 652ms/epoch - 3ms/step
Epoch 6/200
225/225 - 1s - loss: 0.4458 - accuracy: 0.8065 - val_loss: 0.4056 - val_accuracy: 0.8250 - 630ms/epoch - 3ms/step
Epoch 7/200
225/225 - 1s - loss: 0.4408 - accuracy: 0.8114 - val_loss: 0.4008 - val_accuracy: 0.8288 - 602ms/epoch - 3ms/step
Epoch 8/200
225/225 - 1s - loss: 0.4338 - accuracy: 0.8103 - val_loss: 0.3979 - val_accuracy: 0.8300 - 596ms/epoch - 3ms

Epoch 66/200
225/225 - 1s - loss: 0.3910 - accuracy: 0.8376 - val_loss: 0.3481 - val_accuracy: 0.8512 - 665ms/epoch - 3ms/step
Epoch 67/200
225/225 - 1s - loss: 0.3935 - accuracy: 0.8372 - val_loss: 0.3455 - val_accuracy: 0.8512 - 617ms/epoch - 3ms/step
Epoch 68/200
225/225 - 1s - loss: 0.3914 - accuracy: 0.8336 - val_loss: 0.3449 - val_accuracy: 0.8500 - 619ms/epoch - 3ms/step
Epoch 69/200
225/225 - 1s - loss: 0.3935 - accuracy: 0.8317 - val_loss: 0.3460 - val_accuracy: 0.8537 - 610ms/epoch - 3ms/step
Epoch 70/200
225/225 - 1s - loss: 0.3923 - accuracy: 0.8333 - val_loss: 0.3449 - val_accuracy: 0.8537 - 621ms/epoch - 3ms/step
Epoch 71/200
225/225 - 1s - loss: 0.3880 - accuracy: 0.8374 - val_loss: 0.3444 - val_accuracy: 0.8562 - 621ms/epoch - 3ms/step
Epoch 72/200
225/225 - 1s - loss: 0.3934 - accuracy: 0.8353 - val_loss: 0.3460 - val_accuracy: 0.8525 - 602ms/epoch - 3ms/step
Epoch 73/200
225/225 - 1s - loss: 0.3890 - accuracy: 0.8388 - val_loss: 0.3437 - val_accuracy: 0.8575 - 580ms/e

Epoch 131/200
225/225 - 1s - loss: 0.3886 - accuracy: 0.8351 - val_loss: 0.3375 - val_accuracy: 0.8587 - 604ms/epoch - 3ms/step
Epoch 132/200
225/225 - 1s - loss: 0.3897 - accuracy: 0.8367 - val_loss: 0.3388 - val_accuracy: 0.8600 - 597ms/epoch - 3ms/step
Epoch 133/200
225/225 - 1s - loss: 0.3885 - accuracy: 0.8372 - val_loss: 0.3388 - val_accuracy: 0.8637 - 608ms/epoch - 3ms/step
Epoch 134/200
225/225 - 1s - loss: 0.3828 - accuracy: 0.8394 - val_loss: 0.3372 - val_accuracy: 0.8625 - 578ms/epoch - 3ms/step
Epoch 135/200
225/225 - 1s - loss: 0.3891 - accuracy: 0.8339 - val_loss: 0.3389 - val_accuracy: 0.8612 - 525ms/epoch - 2ms/step
Epoch 136/200
225/225 - 1s - loss: 0.3882 - accuracy: 0.8369 - val_loss: 0.3395 - val_accuracy: 0.8600 - 557ms/epoch - 2ms/step
Epoch 137/200
225/225 - 1s - loss: 0.3840 - accuracy: 0.8374 - val_loss: 0.3378 - val_accuracy: 0.8637 - 528ms/epoch - 2ms/step
Epoch 138/200
225/225 - 1s - loss: 0.3868 - accuracy: 0.8374 - val_loss: 0.3368 - val_accuracy: 0.8625 -

225/225 - 1s - loss: 0.3810 - accuracy: 0.8388 - val_loss: 0.3361 - val_accuracy: 0.8600 - 636ms/epoch - 3ms/step
Epoch 196/200
225/225 - 1s - loss: 0.3840 - accuracy: 0.8394 - val_loss: 0.3362 - val_accuracy: 0.8600 - 632ms/epoch - 3ms/step
Epoch 197/200
225/225 - 1s - loss: 0.3872 - accuracy: 0.8396 - val_loss: 0.3361 - val_accuracy: 0.8637 - 623ms/epoch - 3ms/step
Epoch 198/200
225/225 - 1s - loss: 0.3831 - accuracy: 0.8428 - val_loss: 0.3357 - val_accuracy: 0.8625 - 639ms/epoch - 3ms/step
Epoch 199/200
225/225 - 1s - loss: 0.3847 - accuracy: 0.8389 - val_loss: 0.3359 - val_accuracy: 0.8637 - 622ms/epoch - 3ms/step
Epoch 200/200
225/225 - 1s - loss: 0.3848 - accuracy: 0.8424 - val_loss: 0.3377 - val_accuracy: 0.8625 - 626ms/epoch - 3ms/step


In [18]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

[[0.28197938]
 [0.2570681 ]
 [0.12731096]
 [0.0575752 ]
 [0.18797627]]


In [19]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [20]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1542   53]
 [ 235  170]]


In [21]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

85.6 % of data was classified correctly
